## Standard install

In [ ]:
%%capture
!pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu121
!pip install transformers
!pip install wandb huggingface_hub datasets hf_transfer scipy
!pip install numpy==1.26.4

In [ ]:

# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-q6_yc5l1/unsloth_ed8efd84348c44169bb1406e1b1937c5
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-q6_yc5l1/unsloth_ed8efd84348c44169bb1406e1b1937c5
  Resolved https://github.com/unslothai/unsloth.git to commit f1dc078c266b914d8fe652c893ed94f3b35919c8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.1 MB/s eta 0:00:00
  Created wheel for unsloth: filename=unsloth-2024.7-py3-none-any.whl size=127461 sha256=ba2af53504ffa55503922a3fa568011fa4322608a4670fd89c33dd093e333418
  Stored in directory: /tmp/pip-ephem-wheel-cache-ohb8fwxm/wheels/ed/d4/e9/76fb290ee

In [ ]:
import torch
# Determine CUDA version and Pytorch version
cuda_version = torch.version.cuda
pytorch_version = torch.__version__
major_version, minor_version = torch.cuda.get_device_capability()
print(f"CUDA version: {cuda_version}")
print(f"PyTorch version: {pytorch_version}")
print(f"CUDA Device compute capability: {major_version}.{minor_version}")

CUDA version: 12.1
PyTorch version: 2.3.0+cu121
CUDA Device compute capability: 7.5


## Logins

In [ ]:

import wandb
APIKEY = ""
wandb.login(key=APIKEY)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# load the model
cache_dir = ''
model_id = 'Rajeeb321/Qwen_7B_16bit_2'

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 32768 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    cache_dir = cache_dir,
    token = "hf_", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

config.json:   0%|          | 0.00/742 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Qwen2 patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


config.json:   0%|          | 0.00/742 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

## Load Checks

In [ ]:
# Check there are no parameters overflowing onto cpu (meta)
for n, p in model.named_parameters():
  if p.device.type == "meta":
    print(f"{n} is on meta")

In [ ]:
# maximum postional embedding and  EOS token id
print(model.config.max_position_embeddings)
print(model.config.eos_token_id)

32768
151645


In [ ]:
## Set up tokenizer and Padding

In [ ]:
print("TOKENIZER:\n------------",tokenizer,"\n")
print("\nVocab_SIZE_OF_TOKENIZER:\n------------",tokenizer.vocab_size,"\n")

TOKENIZER:
------------ Qwen2TokenizerFast(name_or_path='Rajeeb321/Qwen_7B_16bit_2', vocab_size=151643, model_max_length=32768, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
} 


Vocab_SIZE_OF_TOKENIZER:
------------ 151643 



In [ ]:
print(tokenizer.bos_token)
print(tokenizer.eos_token)
print(tokenizer.pad_token)

None
<|im_end|>
<|endoftext|>


## if not using unsloth and no pad token in model

In [ ]:
# # Optional: automatically done by Unsloth
# pad_tokien =
# if pad_tokien not in tokenizer.get_vocab():
#   print(f"{pad_token} token (pad_token not in tokenizer, adding a {pad_token} token)")
#   # Add the pad token
#   tokenizer.add_token([pad_token], special_token = True)
#   # Set the pad token
#   tokenizer.pad_token_id =  pad_token
# else:
#   print(f"{pad_token} token (pad_token already in tokenizer) and current pad token is {tokenizer.pad_token}")

In [ ]:
# model.pad_token_id = tokenizer.pad_token_id
# model.config.pad_token_id = tokenizer.pad_token_id

# # Check if they are equal
# assert model.pad_token_id == tokenizer.pad_token_id, "The model's pad token ID doesnot match the tokenizer's pad token id"

In [ ]:
# # Print the pad token ids
# print(f"Model pad token id: {model.pad_token_id}")
# print(f"Tokenizer pad token id: {tokenizer.pad_token_id}")
# print(f"Model config pad token Id {model.config.pad_token_id}")
# print(f"Number of  tokens in tokenizer", tokenizer.vocab_size)

In [ ]:
# print("Special Tokens map: ",tokenizer.special_tokens_map)
# print("All special tokens: ", tokenizer.all_special_tokens)

In [ ]:
# # Reiseze the token embeddings if any tokens added
# # model.resize_token_embeddings(tokenizer.vocab_size)
# model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=16)
# print("Number of tokens now in tokenizer: ", tokenizer.vocab_size)

## Chat template

In [ ]:
# Test chat template
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
print(inputs)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Give me a short introduction to large language model.<|im_end|>
<|im_start|>assistant



In [ ]:
# Test tokenization without the chat template
test_prompt = "TEST prompt"

tokens = tokenizer.encode(test_prompt, add_special_tokens=True)
print(tokens)

print(tokenizer.decode(tokens))

[10033, 9934]
TEST prompt


In [ ]:
# tokenizer.padding_side = 'left' # donot use in unsloth

In [ ]:
# Check the integrity of the model and tokenizer after modifying the tokenizer

# Get the vocabulary sizes of the tokenizer
tokenizer_vocab_size = len(tokenizer)
model_embedding_size = model.get_input_embeddings().weight.shape[0]

# Compare the vocabulary size
if tokenizer_vocab_size == model_embedding_size:
  print("The tokenizer vocabulary size matches the model embedding size.")
else:
  print("The tokenizer vocabulary size does not match the model embedding size.")

# Print the size for verification
print(f"Tokenizer vocabulary size: {tokenizer_vocab_size}")
print(f"Model embedding size: {model_embedding_size}")

The tokenizer vocabulary size does not match the model embedding size.
Tokenizer vocabulary size: 151646
Model embedding size: 152064


## Prepare for LoRA fine-tuning

In [ ]:
print(model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNor

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 256, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    # layers_to_transform = [15], # to target only one layer
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA, basically increase learning rate
    modules_to_save = ["embed_tokens", "lm_head"] # making embed_token and lm_head trainable. fully tainable, here it isnot by lora
)

Unsloth: Offloading input_embeddings to disk to save VRAM
Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2024.7 patched 28 layers with 0 QKV layers, 28 O layers and 28 MLP layers.


Unsloth: Casting embed_tokens to float32
Unsloth: Casting lm_head to float32


## Check Trainable paramters

In [ ]:
model.print_trainable_parameters()

trainable params: 1,735,917,568 || all params: 9,351,534,080 || trainable%: 18.5629


## Load the Dataset

In [ ]:
import json
from datasets import load_dataset, concatenate_datasets, DatasetDict
# from transformers import AutoTokenizer

# load the datasets
base_dataset_name_0 = "Rajeeb321/Nepali_news_2_valid"
base_dataset_name_1 = "Rajeeb321/Nepali_news_3_all"
base_dataset_name_2 = "Rajeeb321/Nepali_news_1"
base_dataset_sabdakosh_0 = "Rajeeb321/Nepali_sabdakosh_1"
# blend_dataset_name_0 = "Telugu-LLM-Labs/nepali_alpaca_yahma_cleaned_filtered"
# blend_dataset_name_1 = "mlabonne/orpo-dpo-mix-40k"

# load the base datasets
base_dataset_0 = load_dataset(base_dataset_name_0, split="train", cache_dir = cache_dir, token ="hf_" )
base_dataset_1 = load_dataset(base_dataset_name_1, split="train", cache_dir = cache_dir, token ="hf_")
base_dataset_2 = load_dataset(base_dataset_name_2, split="train", cache_dir = cache_dir, token ="hf_")
base_dataset_sabdakosh_0 = load_dataset(base_dataset_sabdakosh_0, split="train", cache_dir = cache_dir, token ="hf_")
base_dataset_nepali_poem = load_dataset('Rajeeb321/Nepali_poem_laxmi_prasad_devkota', split="train", cache_dir = cache_dir, token ="hf_")
base_dataset_nepali_chat = load_dataset('Rajeeb321/Nepali_Group_Chat', split="train", cache_dir = cache_dir, token ="hf_")


In [ ]:
alpaca_prompt_news = """Below is a News Headline, paired the with News article in Nepali.

### News Headline:
{}

### News article:
{}
"""

alpaca_prompt_sabdakosh = """Below is a Word in Nepali language, paired with meanings of the given word  in Nepali language.

### word:
{}

### Meanings:
{}
"""

alpaca_prompt_nepali_chat = """Below are the messages of a group chat where each is arranged in chronological order. Here each new line represents new message.  Each chat has a user_id and his or her message which is in Nepali language.

### Chat Messages:
{}
"""

alpaca_prompt_nepali_poem = """Below is the portion of a Nepali poem.

### portion of a poem:
{}
"""


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
# EOS_TOKEN = "end"


def formatting_prompts_func_news(examples):
    instructions = examples["Topic"]
    inputs       = examples["News"]
    texts = []
    for instruction, input in zip(instructions, inputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt_news.format(instruction, input) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

def formatting_prompts_func_sabdakosh(examples):
    instructions = examples["word"]
    inputs       = examples["senses"]
    texts = []
    for instruction, input in zip(instructions, inputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt_sabdakosh.format(instruction, input) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass
def formatting_prompts_func_nepali_chat(examples):
    instructions = examples["chat"]
    texts = []
    for instruction in zip(instructions):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt_nepali_chat.format(instruction) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass
def formatting_prompts_func_nepali_poem(examples):
    instructions = examples["poem_portion"]
    texts = []
    for instruction in zip(instructions):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt_nepali_poem.format(instruction) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass



In [ ]:
base_dataset_0 = base_dataset_0.map(formatting_prompts_func_news, batched = True,)
base_dataset_1 = base_dataset_1.map(formatting_prompts_func_news, batched = True,)
base_dataset_2 = base_dataset_2.map(formatting_prompts_func_news, batched = True,)
base_dataset_sabdakosh_0 = base_dataset_sabdakosh_0.map(formatting_prompts_func_sabdakosh, batched = True,)
base_dataset_nepali_chat = base_dataset_nepali_chat.map(formatting_prompts_func_nepali_chat, batched = True,)
base_dataset_nepali_poem = base_dataset_nepali_poem.map(formatting_prompts_func_nepali_poem, batched = True,)



merged_dataset = concatenate_datasets([base_dataset_sabdakosh_0, base_dataset_0, base_dataset_1, base_dataset_2, base_dataset_nepali_chat, base_dataset_nepali_poem  ])

final_datasets = merged_dataset.remove_columns([col for col in merged_dataset.column_names if col not in ["text"]])

# blend_dataset_0 = load_dataset('Telugu-LLM-Labs/nepali_alpaca_yahma_cleaned_filtered', split="train", cache_dir = cache_dir)
# blend_dataset_1 = load_dataset('mlabonne/orpo-dpo-mix-40k', split="train", cache_dir = cache_dir)
# blend_dataset_1 = blend_dataset_1.remove_columns([col for col in blend_dataset_1.column_names if col not in ["chosen"]])

# final_datasets = concatenate_datasets([final_datasets, blend_dataset_0, blend_dataset_1])
final_datasets = final_datasets.shuffle()

In [ ]:
final_datasets

Dataset({
    features: ['text'],
    num_rows: 431283
})

In [ ]:
final_datasets[1]

{'text': None,
 'instruction': None,
 'input': None,
 'output': None,
 'nepali_instruction': None,
 'nepali_input': None,
 'nepali_output': None,
 'chosen': [{'content': 'There is a Nasonex commercial where a 25-30 year old guy is getting into his car, and the CGI generated talking-bee-brand-mascot for Nasonex flys up to him and talks. This is the script:\n\nBEE: Yay, the allergy relief of Nasonex means you can smell your cologne.\n\n(man takes deep breath in)\n\nMAN: Yep\n\nBEE: (A bit snarky - suggesting that his cologne is bad or too much) OK, so that\'s on purpose.\n\n(Man looks befuddled, sniffs himself with a bit of embarrassment, and gets into his car)\n\nBEE: Celebrate all day, all night, all year allergy relief with Nasonex. Now over the counter. Celebrate normal.\n\nIf I was playing the role of the man, what are some extra lines I might add after the bee says "OK, so that\'s on purpose."',
   'role': 'user'},
  {'content': 'As the 25-30 year old man in the Nasonex commercial,

In [ ]:
base_dataset_sabdakosh_0[0]

{'word': 'अ',
 'grammar': 'ना.',
 'senses': "['१. देवनागरी वर्णमालाको स्वर वर्णमध्ये पहिलो स्वर वर्ण; परम्परागत रूपमा कण्ठस्थानबाट उच्चारण हुने ह्रस्व स्वर वर्ण र भाषाविज्ञानअनुसार आधा खुला; केन्द्रीय स्वर वर्ण; लेख्य रूपमा सो स्वर वर्णको प्रतिनिधित्व गर्ने लिपिचिह्न।', '२. लेखाइका क्रममा विषयको विभाजन उपविभाजनका निम्ति स्वर वर्णको प्रयोग गरिँदा दिइने क्रमबोधक पहिलो चिह्न।']",
 'text': "Below is a Word in Nepali language, paired with meanings of the given word  in Nepali language.\n\n### word:\nअ\n\n### Meanings:\n['१. देवनागरी वर्णमालाको स्वर वर्णमध्ये पहिलो स्वर वर्ण; परम्परागत रूपमा कण्ठस्थानबाट उच्चारण हुने ह्रस्व स्वर वर्ण र भाषाविज्ञानअनुसार आधा खुला; केन्द्रीय स्वर वर्ण; लेख्य रूपमा सो स्वर वर्णको प्रतिनिधित्व गर्ने लिपिचिह्न।', '२. लेखाइका क्रममा विषयको विभाजन उपविभाजनका निम्ति स्वर वर्णको प्रयोग गरिँदा दिइने क्रमबोधक पहिलो चिह्न।']\n<|im_end|>"}

In [ ]:
len(final_datasets)

504438

## Use completion only or not

In [ ]:
completions_only = False

In [ ]:
## only if not using alpaca prompt and completion_only = True
# response_template = '<|start_header_id|>assistant<|end_header_id|>'
# if completions_only is True:
#   from trl import DataCollatorForCompletionOnlyLM
#   response_template_with_context = response_template
#   data_collator = DataCollatorForCompletionOnlyLM(response_template_ids, tokenizer=tokenizer)

In [ ]:
if completions_only is False:
  collator = None

## Training

In [ ]:
#@title Trainer parameters
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = final_datasets,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    # packing = True, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio=0.03,
        max_steps = 60,
        learning_rate = 5e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "unsupervised_qwen_7b_news_sabdakosh_poem_chat_dir",
        report_to = "wandb"
    ),
)

Map (num_proc=2):   0%|          | 0/431283 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")


GPU = Tesla T4. Max memory = 14.748 GB.
13.428 GB of memory reserved.


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
#@title Train
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 431,283 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 1,735,917,568
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: donrajeep321 (rajeeb). Use `wandb login --relogin` to force relogin


OutOfMemoryError: CUDA out of memory. Tried to allocate 130.00 MiB. GPU 

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

## Plot the graph

In [ ]:
import matplotlib.pyplot as plt

# Initialize list to hold training and evaluation losses and steps
train_losses = []
eval_losses = []
train_step = []
eval_step = []

#Populate the list from log history
#import pandas as pd
# pd.DataFrame(trainer.state.log_history)
for entry in trainer.state.log_history:
  if 'loss' in entry:
    train_losses.append(entry['loss'])
    train_step.append(entry['step'])
  if 'eval_loss' in entry:
    eval_losses.append(entry['eval_loss'])
    eval_step.append(entry['step'])

# plot the losses
plt.plot(train_step, train_losses, label = 'Train Loss')
plt.plot(eval_step, eval_losses, label = 'eval Loss')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.legend()
plt.show()

## Save the model

In [ ]:
#@title Lora weights
model.push_to_hub("Rajeeb321/", token = "...") # Online saving
tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

In [ ]:
#@title Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

In [ ]:
#@title Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

In [ ]:
#@title Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")